# Case study 1：添加 Prefetcher && Case Study 1: Adding a Prefetcher

在这一部分，我们将通过一个在 XS-GEM5 中添加新预取器的示例，演示如何在 XS-GEM5 上实现并评估新的特性。

In this section, we present an example of adding a new prefetcher to XS-GEM5, demonstrating how to implement and evaluate new features on the XS-GEM5 platform.

我们先创建本节的工作目录。

First, we create the working directory for this section.

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${WORK_DIR}/04-gem5/04-case-study-prefetcher

这里我们运行这个 Bash 脚本，用于创建 04-case-study-prefetcher 这个文件目录。

Here we run this Bash script to create the 04-case-study-prefetcher directory.

和 01-compile 以及 02-first-run 一样，我们先编译运行一份原始的 GEM5 作为基线。

Just like in 01-compile and 02-first-run, we first build and run a baseline version of GEM5.

In [ ]:
%%bash
cd .. && source env.sh
cd ${GEM5_HOME}

#git reset --hard 30be231238bc0f9d3687efd59826748c02ad5f7b

# 编译
# compile
# scons build/RISCV/gem5.opt -j8

# GEM5=${WORK_DIR}/04-gem5/04-case-study-prefetcher/gem5-base.opt
# cp build/RISCV/gem5.opt ${GEM5}

# 运行
# run
export LD_LIBRARY_PATH=$(get_asset gem5-precompile):${LD_LIBRARY_PATH}
export GCBV_REF_SO=$(get_asset gem5-precompile/riscv64-nemu-interpreter-c1469286ca32-so)

mkdir -p ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base
cd ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base

$(get_asset gem5-precompile/gem5-base.opt) ${GEM5_HOME}/configs/example/xiangshan.py \
    --generic-rv-cpt=$(get_asset workload/mcf_12253_0.137576_.zstd) \
    --gcpt-restorer=$(get_asset workload/normal-gcb-restorer.bin) \
    -I 300000

这里我们使用这个 Bash 脚本来编译 XS-GEM5，并运行 mcf workload 的切片。

运行生成的文件会保存在 result_base 这个文件夹中。

如果大家在过程中遇到问题，可以举手，我们在场的小伙伴会协助大家。

编译完成后，我们就能得到 baseline 配置下运行 mcf 切片的结果。


Here we use this Bash script to compile XS-GEM5 and run the mcf workload slice.

The generated files will be stored in the result_base folder.

If anyone encounters any issues, please raise your hand and our team members will assist you.

After the compilation is completed, we will obtain the baseline results of running the mcf slice.

我们准备好了一个实现 Despacito Stream Prefetcher 的 patch 文件 `04-DespacitoStream.patch`，现在可以将它应用到 GEM5 上并再次编译运行。


We have prepared a patch file '04-DespacitoStream.patch', which implements the Despacito Stream Prefetcher. Now we can apply it to GEM5 and rebuild the project.

In [ ]:
%%bash
cd .. && source env.sh
cd ${GEM5_HOME}

#git reset --hard 30be231238bc0f9d3687efd59826748c02ad5f7b

# 应用 prefetcher patch
# use prefetcher patch
#git config user.name "xiangshan-bootcamp"
#git config user.email "xiangshan-bootcamp@example.com"
#git am -3 ${XS_PROJECT_ROOT}/04-gem5/04-DespacitoStream.patch

# 编译
# compile
#scons build/RISCV/gem5.opt -j8

#GEM5=${WORK_DIR}/04-gem5/04-case-study-prefetcher/gem5-despacito.opt
#cp build/RISCV/gem5.opt ${GEM5}

# 运行
# run
export LD_LIBRARY_PATH=$(get_asset gem5-precompile):${LD_LIBRARY_PATH}
export GCBV_REF_SO=$(get_asset gem5-precompile/riscv64-nemu-interpreter-c1469286ca32-so)

mkdir -p ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito
cd ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito

# {GEM%}
$(get_asset gem5-precompile/gem5-dsprefetch.opt) ${GEM5_HOME}/configs/example/xiangshan.py \
    --generic-rv-cpt=$(get_asset workload/mcf_12253_0.137576_.zstd) \
    --gcpt-restorer=$(get_asset workload/normal-gcb-restorer.bin) \
    -I 300000

这里我们使用这个 Bash 脚本来编译已添加补丁的 XS-GEM5，并运行 mcf workload 的切片。

同样地，XS-GEM5 运行后生成的文件会保存在 result_despacito 文件夹中。

如果大家在过程中遇到问题，可以举手，我们在场的小伙伴会协助大家。

编译完成后，我们将得到添加补丁后的 XS-GEM5 运行 mcf 切片的结果。


Here we use this Bash script to compile the patched version of XS-GEM5 and run the mcf workload slice.

Similarly, the generated files from XS-GEM5 will be stored in the result_despacito folder.

If anyone encounters any issues, please raise your hand and our team members will assist you.

After the compilation is completed, we will obtain the results of running the mcf slice with the patched XS-GEM5.

最后，我们用一个简单的脚本对比一下添加预取器前后的差异。

Finally, we use a simple script to compare the differences before and after adding the prefetcher.

In [ ]:
%%bash
cd .. && source env.sh

echo -n "Baseline                IPC: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2
echo -n "Despacito Stream        IPC: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2
echo -n "Baseline         L1D Misses: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2
echo -n "Despacito Stream L1D Misses: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2

这里展示了添加预取器前后 IPC 和 L1D Misses 的变化情况。

可以看到，在添加新预取器之后，IPC 从 0.30 提升到 0.36，增加了 0.06。

与此同时，L1D Misses 从 203,950 下降到 137,700，减少了 66,250 次。


This figure shows the change in IPC and L1D Misses before and after adding the prefetcher.

We can see that after introducing the new prefetcher, IPC increased from 0.30 to 0.36, an improvement of 0.06.

At the same time, L1D Misses decreased from 203,950 to 137,700, a reduction of 66,250.